<a href="https://colab.research.google.com/github/Alro10/PyTorch1.xTutorials/blob/master/lesson03-Logistic-Regression/03_logistic_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Linear Regression

In this tutorial, we will take a look and implement the classic linear regression algorithm 

$$y = a + bx + \varepsilon$$

Linear Regression involves creating a best fit linear line such the distance between the y' points on line and real values is minimum (the summation of the distance is known as our loss)

- The code is based on:
  - https://github.com/yunjey/pytorch-tutorial/tree/master/tutorials/01-basics/logistic_regression
  - https://www.kaggle.com/negation/pytorch-logistic-regression-tutorial

## Gradient Descent

### Step 1: Compute the loss

$$MSE = \frac{1}{N} \sum_{i=1}^N (y_i - \hat{y}_i)^2 $$

$$MSE = \frac{1}{N} \sum_{i=1}^N (y_i - a - bx_i )^2 $$

<h4><center>Mean Square Error (MSE)</center></h4>


### Step 2: Compute the gradients

\begin{equation}
\frac{\partial MSE}{\partial a} = \frac{\partial MSE}{\partial \hat{y}_i} \frac{\partial  \hat{y}_i}{\partial a}  = \frac{1}{N} \sum_{i=1}^N 2(y_i  - a - bx_i) (-1) = -2\frac{1}{N} \sum_{i=1}^N (y_i - \hat{y}_i)
\end{equation}


\begin{equation}
\frac{\partial MSE}{\partial a} = \frac{\partial MSE}{\partial \hat{y}_i} \frac{\partial  \hat{y}_i}{\partial a}  = \frac{1}{N} \sum_{i=1}^N 2(y_i  - a - bx_i) (-1) (x_i)= -2\frac{1}{N} \sum_{i=1}^N x_i(y_i - \hat{y}_i)
\end{equation}

<h4><center>Computing gradients w.r.t. parameters</center></h4>

### Step 3: Update the parameters

\begin{equation}
a = a - \alpha \frac{\partial MSE}{\partial a} 
\end{equation}

\begin{equation}
b = b - \alpha \frac{\partial MSE}{\partial b} 
\end{equation}

### Step 4: Loop for training 

Go back to **Step 1** and restart the process. Repeating the process over and over, for many epochs is training a model

In [0]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms



# Hyper-parameters 
input_size = 784
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

# MNIST dataset (images and labels)
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader (input pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [4]:
torch.cuda.is_available()

True

In [5]:
# Logistic regression model
model = nn.Linear(input_size, num_classes)

# Loss and optimizer
# nn.CrossEntropyLoss() computes softmax internally
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Reshape images to (batch_size, input_size)
        images = images.reshape(-1, 28*28)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 2.2002
Epoch [1/5], Step [200/600], Loss: 2.1121
Epoch [1/5], Step [300/600], Loss: 2.0450
Epoch [1/5], Step [400/600], Loss: 1.9214
Epoch [1/5], Step [500/600], Loss: 1.8055
Epoch [1/5], Step [600/600], Loss: 1.8352
Epoch [2/5], Step [100/600], Loss: 1.6922
Epoch [2/5], Step [200/600], Loss: 1.6418
Epoch [2/5], Step [300/600], Loss: 1.6429
Epoch [2/5], Step [400/600], Loss: 1.5583
Epoch [2/5], Step [500/600], Loss: 1.5216
Epoch [2/5], Step [600/600], Loss: 1.5120
Epoch [3/5], Step [100/600], Loss: 1.5502
Epoch [3/5], Step [200/600], Loss: 1.3964
Epoch [3/5], Step [300/600], Loss: 1.3476
Epoch [3/5], Step [400/600], Loss: 1.3295
Epoch [3/5], Step [500/600], Loss: 1.2904
Epoch [3/5], Step [600/600], Loss: 1.3631
Epoch [4/5], Step [100/600], Loss: 1.2645
Epoch [4/5], Step [200/600], Loss: 1.1891
Epoch [4/5], Step [300/600], Loss: 1.1267
Epoch [4/5], Step [400/600], Loss: 1.2308
Epoch [4/5], Step [500/600], Loss: 1.0688
Epoch [4/5], Step [600/600], Loss:

In [6]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()

    print('Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the model on the 10000 test images: 82 %


In [0]:
# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

In [8]:
!ls

model.ckpt  sample_data
